# Winery classification with the multivariate Gaussian

In this notebook, we return to winery classification, using the full set of 13 features.

## 1. Load in the data 

As usual, we start by loading in the Wine data set. Make sure the file `wine.data.txt` is in the same directory as this notebook.

Recall that there are 178 data points, each with 13 features and a label (1,2,3). As before, we will divide this into a training set of 130 points and a test set of 48 points.

In [89]:
# Standard includes
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
# Useful module for dealing with the Gaussian density
from scipy.stats import norm, multivariate_normal

In [90]:
# Load data set.
data = np.loadtxt('wine.data.txt', delimiter=',')
# Names of features
featurenames = ['Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash','Magnesium', 'Total phenols',
                'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue',
                'OD280/OD315 of diluted wines', 'Proline']
# Split 178 instances into training set (trainx, trainy) of size 130 and test set (testx, testy) of size 48
np.random.seed(0)
perm = np.random.permutation(178)
trainx = data[perm[0:130],1:14]
trainy = data[perm[0:130],0]
testx = data[perm[130:178], 1:14]
testy = data[perm[130:178],0]

## 2. Fit a Gaussian generative model

We now define a function that fits a Gaussian generative model to the data.
For each class (`j=1,2,3`), we have:
* `pi[j]`: the class weight
* `mu[j,:]`: the mean, a 13-dimensional vector
* `sigma[j,:,:]`: the 13x13 covariance matrix

This means that `pi` is a 4x1 array (Python arrays are indexed starting at zero, and we aren't using `j=0`), `mu` is a 4x13 array and `sigma` is a 4x13x13 array.

In [91]:
def fit_generative_model(x,y):
    k = 3  # labels 1,2,...,k
    d = (x.shape)[1]  # number of features
    mu = np.zeros((k+1,d))
    sigma = np.zeros((k+1,d,d))
    pi = np.zeros(k+1)
    for label in range(1,k+1):
        indices = (y == label)
        mu[label] = np.mean(x[indices,:], axis=0)
        sigma[label] = np.cov(x[indices,:], rowvar=0, bias=1)
        pi[label] = float(sum(indices))/float(len(y))
    return mu, sigma, pi

In [92]:
# Fit a Gaussian generative model to the training data
mu, sigma, pi = fit_generative_model(trainx,trainy)

## 3. Use the model to make predictions on the test set

<font color="magenta">**For you to do**</font>: Define a general purpose testing routine that takes as input:
* the arrays `pi`, `mu`, `sigma` defining the generative model, as above
* the test set (points `tx` and labels `ty`)
* a list of features `features` (chosen from 0-12)

It should return the number of mistakes made by the generative model on the test data, *when restricted to the specified features*. For instance, using the just three features 2 (`'Ash'`), 4 (`'Magnesium'`) and 6 (`'Flavanoids'`) results in 7 mistakes (out of 48 test points), so

        `test_model(mu, sigma, pi, [2,4,6], testx, testy)`

should print 7/48.

**Hint:** The way you restrict attention to a subset of features is by choosing the corresponding coordinates of the full 13-dimensional mean and the appropriate submatrix of the full 13x13 covariance matrix.

In [93]:
from math import exp

# Now test the performance of a predictor based on a subset of features
def test_model(mu, sigma, pi, features, tx, ty):
    # print("Mu shape:", mu.shape)
    # print("Sigma shape:", sigma.shape)
    # print("Pi shape:", pi.shape)
    # print("X shape:", tx.shape)
    # print("Y shape:", ty.shape)
    # sigma = np.array([sigma[:, features, :][:, :, features]] * tx.shape[0])
    # mu = np.array([mu[:, features]] * tx.shape[0])
    # x = np.transpose(np.array([tx[:, features]] * mu.shape[1]), axes=[1, 0, 2])
    # print("Mu shape:", mu.shape)
    # print("Sigma shape:", sigma.shape)
    # print("Pi shape:", pi.shape)
    # print("X shape:", x.shape)
    # c = 1 / (((2 * pi) ** (len(features) / 2)) * abs(sigma) ** (1 / 2))
    # print("C shape", c.shape)
    # # p = c * exp(-(1 / 2) * np.transpose(x - mu) * np.negative(sigma) * (x - mu))
    # print(np.transpose(x - mu).shape)
    # print(np.dot(np.negative(sigma[1, 1, :, :]),(x - mu)[1, 1, :]).shape)
    # print(np.dot(np.transpose(np.negative(sigma)), (x - mu)).shape)
    # p = exp(-(1 / 2) * np.transpose(x - mu) * np.negative(sigma) * (x - mu))
    # print(p.shape)

    ### Your code goes here
    sigma = sigma[:, features, :][:, :, features]
    mu = mu[:, features]
    x = tx[:, features]

    p = np.zeros((tx.shape[0], mu.shape[0]))
    for i in range(p.shape[0]):
        for j in range(p.shape[1]):
            if pi[j] != 0:
                c = 1 / (((2 * pi[j]) ** (len(features) / 2)) * np.linalg.det(sigma[j]) ** (1 / 2))
                exp_inside = -(1 / 2) * np.dot(
                    np.dot(np.transpose(x[i] - mu[j]), np.negative(sigma[j])),
                    (x[i] - mu[j])
                )
                pij = c * np.exp(exp_inside)
                p[i][j] = np.prod(pij)

    print(p)
    print([np.argmax(item) for item in p])
    ###


### <font color="magenta">Fast exercises</font>

*Note down the answers to these questions. You will need to enter them as part of this week's assignment.*

Exercise 1. How many errors are made on the test set when using the single feature 'Ash'?

In [94]:
test_model(mu, sigma, pi, [2], testx, testy)

[[0.         6.4144254  3.2982084  8.58353555]
 [0.         6.41426742 3.27423935 8.58139757]
 [0.         6.41141908 3.27940422 8.57943023]
 [0.         6.4240154  3.27686813 8.58983041]
 [0.         6.41567875 3.30241096 8.58494201]
 [0.         6.4240154  3.27686813 8.58983041]
 [0.         6.41187091 3.27715164 8.57962225]
 [0.         6.41882349 3.3118362  8.58837613]
 [0.         6.41523735 3.30097244 8.58445019]
 [0.         6.41325622 3.27487881 8.5805991 ]
 [0.         6.41714473 3.30695309 8.5865555 ]
 [0.         6.42766993 3.33480968 8.59773294]
 [0.         6.41194    3.2876259  8.58055735]
 [0.         6.41187091 3.27715164 8.57962225]
 [0.         6.41187091 3.27715164 8.57962225]
 [0.         6.42595836 3.33059568 8.59594273]
 [0.         6.41663244 3.30540123 8.58599466]
 [0.         6.41187091 3.27715164 8.57962225]
 [0.         6.42139311 3.31887998 8.59112609]
 [0.         6.41567875 3.30241096 8.58494201]
 [0.         6.41164718 3.28568471 8.58014823]
 [0.         

Exercise 2. How many errors when using 'Alcohol' and 'Ash'?

In [95]:
test_model(mu, sigma, pi, [0,2], testx, testy)

[[ 0.         16.3601295   9.65558161 24.73198259]
 [ 0.         20.2955571   6.93136855 24.8107495 ]
 [ 0.         18.25126131  7.15639025 23.3121522 ]
 [ 0.         19.36067279  7.00654141 24.12763212]
 [ 0.         16.35047468  9.45003066 24.49604085]
 [ 0.         17.359053    7.53973567 22.97275197]
 [ 0.         18.73808763  7.0628168  23.61726818]
 [ 0.         16.35593632  9.21916796 24.24761433]
 [ 0.         18.15405553  7.20338873 23.23496053]
 [ 0.         20.62688869  6.92460907 25.07378023]
 [ 0.         16.50015135  8.39935207 23.36430262]
 [ 0.         16.38944008  9.60258793 24.77875259]
 [ 0.         22.94577096  7.01825201 27.01070743]
 [ 0.         16.4171662   8.63767691 23.48773963]
 [ 0.         17.31026333  7.50060412 22.93149635]
 [ 0.         26.1215905   7.39395658 29.75320783]
 [ 0.         20.15161564  6.99085379 24.56917901]
 [ 0.         16.35822249  8.96045344 23.80337484]
 [ 0.         16.50172845  8.4301928  23.41046399]
 [ 0.         16.50676098  8.37

Exercise 3. How many errors when using 'Alcohol', 'Ash', and 'Flavanoids'?

In [96]:
test_model(mu, sigma, pi, [0,2,6], testx, testy)

[[  0.          57.79703909  26.31134984 200.43327457]
 [  0.          86.90748985  11.04115857 150.59815426]
 [  0.          95.10791267  15.05967264 137.42103162]
 [  0.         129.05592739  21.98504182 142.28752707]
 [  0.          56.93507872  22.97299955 191.96485906]
 [  0.          61.3621013   12.46993156 151.79170065]
 [  0.          95.29711688  13.97878033 139.48317361]
 [  0.          56.38095502  16.23673512 170.09445024]
 [  0.         114.6153259   21.0212713  136.66749276]
 [  0.          76.81658388  11.18421141 161.02882388]
 [  0.          58.19116796  13.73457072 156.62588047]
 [  0.          56.29032024  18.35958439 180.58766316]
 [  0.          89.64545813  11.12116375 170.81582822]
 [  0.          56.38470926  18.04209297 173.1364905 ]
 [  0.          78.80915512  13.73320172 136.64542961]
 [  0.         105.03492796  11.70125589 188.03369267]
 [  0.         137.79492022  20.18770156 144.63190131]
 [  0.          56.72783224  15.02337608 161.93584998]
 [  0.    

Exercise 4. How many errors when using all 13 features?

In [97]:
test_model(mu, sigma, pi, range(0,13), testx, testy)

[[ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]
 [ 0. inf inf inf]]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

/Users/kamil.rafalko/.conda/envs/hands-on-machine-learning/lib/python3.7/site-packages/ipykernel_launcher.py:40: RuntimeWarning: overflow encountered in exp


Exercise 5. In lecture, we got somewhat different answers to these questions. Why do you think that might be?